## Elliptic curves

In [232]:
import random

# CONSTANTES

a = 0
b = 3
p = 7

# CONVERSIONS

def invModulaire(a, p):
    return pow(a, p - 2) % p

def fromCartesianToJacobian(P, z):
    return ((P[0] * (z ** 2)) % p, (P[1] * (z ** 3)) % p, z)

def fromJacobianToCartesian(P):
    invZ = invModulaire(P[2], p)
    return ((P[0] * (invZ ** 2)) % p, (P[1] * (invZ ** 3)) % p)

### 1. Opérations sur les points d'une courbe elliptique en coordonnées carthésiennes:

- Inverse modulaire
- Addition
- Doublement
- Appartenance d'un point à la courbe

In [233]:
def addition(P, Q, p):
    lam = ((Q[1] - P[1]) * invModulaire(Q[0] - P[0], 7)) % p
    x_3 = (lam ** 2 - P[0] - Q[0])
    y_3 = (lam * (P[0] - x_3) - P[1])
    return (x_3 % p, y_3 % p)

def doublement(P, p, a):
    lam = ((3 * (P[0] ** 2) + a) * invModulaire(2 * P[1], 7))
    x_2 = (lam ** 2 - 2 * P[0]) % p
    y_2 = (lam * (P[0] - x_2) - P[1]) % p
    return (x_2, y_2)

def onTheCurve(P, p, a, b):
    return ((P[1] ** 2) % p == (P[0] ** 3 + a * P[0] + b) % p)

P = (1, 2)
Q = (4, 5)
add = addition(P, Q, p)
print("[CARTESIAN-] P + Q = " + str(add))
double = doublement(P, p, a)
print("[CARTESIAN-]    2P = " + str(double))

[CARTESIAN-] P + Q = (3, 3)
[CARTESIAN-]    2P = (6, 3)


### 2. Opérations sur les points d'une courbe elliptique en coordonnées jacobiennes:

- Addition
- Doublement
- Appartenance d'un point à la courbe

In [234]:
def addition_jacob(P, Q, p):
    Z1Z1 = P[2] ** 2
    Z2Z2 = Q[2] ** 2
    U1 = P[0] * Z2Z2
    U2 = Q[0] * Z1Z1
    S1 = P[1] * Q[2] * Z2Z2
    S2 = Q[1] * P[2] * Z1Z1
    H = U2 - U1
    I = (2 * H) ** 2
    J = H * I
    r = 2 * (S2 - S1)
    V = U1 * I
    X3 = (r ** 2 - J - 2 * V) % p
    return (X3, (r * (V - X3) - 2 * S1 * J) % p, (((P[2] + Q[2]) ** 2 - Z1Z1 - Z2Z2) * H) % p)

def doublement_jacob(P, p):
    A = P[0] ** 2
    B = P[1] ** 2
    C = B ** 2
    D = 2 * ((P[0] + B) ** 2 - A - C)
    E = 3 * A
    F = E ** 2
    X3 = (F - 2 * D) % p
    return (X3, (E * (D - X3) - 8 * C) % p, (2 * P[1] * P[2]) % p)

def onTheCurve(P, p, a, b):
    return ((P[1] ** 2) % p == (P[0] ** 3 + a * P[0] * (P[2] ** 4) + b * (P[2] ** 6)) % p)

P_tilde = (5, 3, 4)
Q_tilde = (1, 2, 1)
add_tilde = addition_jacob(P_tilde, Q_tilde, p)
print("[JACOBIAN--] P + Q = " + str(add_tilde))
double_tilde = doublement_jacob(P_tilde, p)
print("[JACOBIAN--]    2P = " + str(double_tilde))

[JACOBIAN--] P + Q = (4, 2, 4)
[JACOBIAN--]    2P = (1, 2, 3)


### 3. Exponentiation rapide

In [235]:
def valueBit(x, i):
    return (x >> i) & 1

def nbBits(x):
    return len(bin(x)[2:])

def exponentiation(a, k, p):
    n = nbBits(k)
    result = 1
    for i in range(0, n):
        if (valueBit(k, n - i) == 1):
            result = result * a
        result = result ** 2
    return result % p

def exponentiation_stylee(a, k, p):
    n = nbBits(k)
    result = 1
    for i in range(0, n):
        result = (result ** 2) * a ** (k >> n - i) & 1
    return result % p

print(exponentiation(a, 4, p))

0


### 4. Multiplication par un scalaire d'un point en coordonnées jacobiennes

In [236]:
def multiplication_scalaire(P, k, p):
    n = nbBits(k)
    result = P
    for i in range(0, n - 1):
        result = doublement_jacob(result, p)
        if (valueBit(k, n - i) == 1):
            result = addition_jacob(result, P, p)
    return result

P = (1, 2, 1)
k = 3
multiplication_3 = multiplication_scalaire(P, k, p)
print("[JACOBIAN--] " + str(k) + "P = " + str(multiplication_3))

[JACOBIAN--] 3P = (5, 3, 4)


### 5. Echelle de Montgomery

In [237]:
def echelleMontgomery(P, k, p):
    n = nbBits(k)
    R = [P, doublement(P, p, a)]
    for i in range(n - 2, -1, -1):
        b = valueBit(k, i)
        R[1 - b] = addition(R[1 - b], R[b], p)
        R[b] = doublement(R[b], p, a)
    return R[0]

def echelleMontgomeryJacob(P, k, p):
    n = nbBits(k)
    R = [P, doublement_jacob(P, p)]
    for i in range(n - 2, -1, -1):
        b = valueBit(k, i)
        R[1 - b] = addition_jacob(R[1 - b], R[b], p)
        R[b] = doublement_jacob(R[b], p)
    return R[0]

P = (1, 5, 1)
k = 2
R_returned = echelleMontgomeryJacob(P, k, p)
print("[JACOBIAN--] " + str(k) + "P = " + str(R_returned))
print("[CARTESIAN-] " + str(k) + "P = " + str(fromJacobianToCartesian(R_returned)))

[JACOBIAN--] 2P = (5, 3, 3)
[CARTESIAN-] 2P = (6, 4)


In [238]:
def generateRandomInt(p):
    return random.randint(1, p - 1)

print(generateRandomInt(p))

2


### 6. Algorithme de Diffie-Hellman

In [239]:
def DiffieHellman(P, p, a, b, k):
    n1 = generateRandomInt(k)
    print("[ALICE-] Génération d'un nombre aléatoire entre 1 et " + str(k) + " : n1 = " + str(n1))
    n2 = generateRandomInt(k)
    print("[BOB---] Génération d'un nombre aléatoire entre 1 et " + str(k) + " : n2 = " + str(n2))
    A = echelleMontgomery(P, n1, p)
    print("[ALICE-] Calcul de A = n1 * P = " + str(A))
    B = echelleMontgomery(P, n2, p)
    print("[BOB---] Calcul de B = n2 * P = " + str(B))
    return (echelleMontgomery(B, n1, p), echelleMontgomery(A, n2, p))

P = (5, 3)
k = 13
print("[ALICE-] Création d'une clé pour une communication avec Bob")
print("[ALICE-] Envoi de P = " + str(P) + ", point d'ordre k = " + str(k) + " qui appartient à la courbe elliptique (a, b) = (" + str(a) + ", " + str(b) + ")")
(C1, C2) = DiffieHellman(P, p, a, b, k)
print("[ALICE-] Calcul de C1 = " + str(C1))
print("[BOB---] Calcul de C2 = " + str(C2))

[ALICE-] Création d'une clé pour une communication avec Bob
[ALICE-] Envoi de P = (5, 3), point d'ordre k = 13 qui appartient à la courbe elliptique (a, b) = (0, 3)
[ALICE-] Génération d'un nombre aléatoire entre 1 et 13 : n1 = 8
[BOB---] Génération d'un nombre aléatoire entre 1 et 13 : n2 = 8
[ALICE-] Calcul de A = n1 * P = (4, 2)
[BOB---] Calcul de B = n2 * P = (4, 2)
[ALICE-] Calcul de C1 = (5, 4)
[BOB---] Calcul de C2 = (5, 4)
